In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/unstructured/llm_data.csv')

In [5]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
# from transformers import Trainer, TrainingArguments

#ref-> https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/language_modeling.ipynb


In [4]:
# !pip uninstall bitsandbytes -y

In [5]:
# !pip uninstall bitsandbytes -y

In [6]:
# !pip install bitsandbytes==0.41.0

In [17]:
# !pip freeze

In [3]:
from transformers import Trainer, TrainingArguments

/home/agoparaj/.conda/envs/nlp2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

model_name = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [9]:
tokenized_dataset = df.text.apply(tokenizer)

In [10]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [11]:

training_args = TrainingArguments(
    output_dir='..data/meta/distil_roberta_chkpts/',
    # evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    
    data_collator=data_collator,
)

In [13]:
trainer.train()
#takes around 40 minutes with one gpu

/home/agoparaj/.conda/envs/nlp2/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.803900
1000,1.566200
1500,1.522000
2000,1.456500
2500,1.440600
3000,1.353500
3500,1.375300
4000,1.334200
4500,1.331000
5000,1.270700


TrainOutput(global_step=54537, training_loss=1.0041980032047637, metrics={'train_runtime': 2289.9212, 'train_samples_per_second': 190.526, 'train_steps_per_second': 23.816, 'total_flos': 1517591332832256.0, 'train_loss': 1.0041980032047637, 'epoch': 3.0})

In [15]:
model.save_pretrained('../data/meta/model-distilrobertabase/')

In [7]:
m1 = AutoModelForMaskedLM.from_pretrained('../data/meta/model-distilrobertabase/')
t1 =  AutoTokenizer.from_pretrained("distilroberta-base")
#we are using the tokenizer from distilroberta-base

In [11]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='../data/meta/model-distilrobertabase/',tokenizer = 'distilroberta-base')
unmasker("db00014 is similar to db<mask>.")



[{'score': 0.06373988837003708,
  'token': 33279,
  'token_str': '015',
  'sequence': 'db00014 is similar to db015.'},
 {'score': 0.053195249289274216,
  'token': 38049,
  'token_str': '006',
  'sequence': 'db00014 is similar to db006.'},
 {'score': 0.049640484154224396,
  'token': 151,
  'token_str': '000',
  'sequence': 'db00014 is similar to db000.'},
 {'score': 0.04174694046378136,
  'token': 37129,
  'token_str': '013',
  'sequence': 'db00014 is similar to db013.'},
 {'score': 0.03956039249897003,
  'token': 37277,
  'token_str': '008',
  'sequence': 'db00014 is similar to db008.'}]

In [81]:
# we can see that the tokenizer is tokenizing the compound name which is undesirable and then it is
#focusing on the english words 'to' and 'similar' more than the compound name.

In [12]:
unmasker("side effect c0038990 caused by db<mask>")

[{'score': 0.9823749661445618,
  'token': 4,
  'token_str': '.',
  'sequence': 'side effect c0038990 caused by db.'},
 {'score': 0.0016314154490828514,
  'token': 2663,
  'token_str': '01',
  'sequence': 'side effect c0038990 caused by db01'},
 {'score': 0.0014089683536440134,
  'token': 612,
  'token_str': '00',
  'sequence': 'side effect c0038990 caused by db00'},
 {'score': 0.0007700097048655152,
  'token': 38049,
  'token_str': '006',
  'sequence': 'side effect c0038990 caused by db006'},
 {'score': 0.0007663475116714835,
  'token': 19130,
  'token_str': '002',
  'sequence': 'side effect c0038990 caused by db002'}]

In [13]:
# we can see that the model is giving random outputs which are not relevant and useful.
# i think i should have trained it from scratch instead of finetuning as this is a very domain specefic task and the models' 
# previous knowledge is not exactly useful.
# i could have used a clinical model for this task but it might take a lot of time to pre-train and use a larger model.

In [14]:
unmasker("db00014 is similar to db<mask>.")

[{'score': 0.06373988837003708,
  'token': 33279,
  'token_str': '015',
  'sequence': 'db00014 is similar to db015.'},
 {'score': 0.053195249289274216,
  'token': 38049,
  'token_str': '006',
  'sequence': 'db00014 is similar to db006.'},
 {'score': 0.049640484154224396,
  'token': 151,
  'token_str': '000',
  'sequence': 'db00014 is similar to db000.'},
 {'score': 0.04174694046378136,
  'token': 37129,
  'token_str': '013',
  'sequence': 'db00014 is similar to db013.'},
 {'score': 0.03956039249897003,
  'token': 37277,
  'token_str': '008',
  'sequence': 'db00014 is similar to db008.'}]

In [15]:
# we can see the tokenizer is at the blame again, it is removing the zeros to the left which is not desirable.we should not split 
# the ids and should wor at word level
# we have to make a custom tokneizer to prevent this.

In [16]:
# I did not find LLMs to be useful for this task because of the nature of the dataset, its not textual and does not include notes or comments
# to help the model understand from a bigger pool of data and to provide a need to use LLM instead of machine learning models.